# RAG vs. GraphRAG: Final Answer Comparison

This notebook demonstrates the difference in final answers generated by **Standard Vector RAG** and **Semantica GraphRAG** for a complex multi-hop query. We use **FalkorDB** as our high-performance graph backend.

### FalkorDB Setup
To use the persistent graph store, you can start a FalkorDB instance using the provided Docker Compose file:
```bash
docker compose up -d
```

In [ ]:
%pip install -qU semantica networkx matplotlib plotly pandas faiss-cpu falkordb

## 1. Environment & Store Initialization
We set up our **Vector Store** for semantic search and **FalkorDB** for persistent graph storage.

In [ ]:
from semantica.core import Semantica
from semantica.vector_store import VectorStore
from semantica.graph_store import GraphStore
from semantica.kg import GraphBuilder
from semantica.split import TextSplitter
from semantica.normalize import TextNormalizer

# 1. Core Semantica & Vector Store
v_core = Semantica()
vs = VectorStore(backend="faiss", dimension=1536)

# 2. FalkorDB Persistent Graph Store
graph_store = GraphStore(
    backend="falkordb",
    host="localhost",
    port=6379,
    graph_name="intelligence_graph"
)

try:
    graph_store.connect()
    print("Successfully connected to FalkorDB backend.")
    use_persistent = True
except Exception as e:
    print(f"FalkorDB connection failed: {e}. Falling back to in-memory mode.")
    use_persistent = False

# 3. Graph Builder with Persistence Support
gb = GraphBuilder(merge_entities=True, graph_store=graph_store if use_persistent else None)
splitter = TextSplitter(method="recursive", chunk_size=800, chunk_overlap=100)
normalizer = TextNormalizer()

## 2. Real-Time Intelligence Ingestion
We pull live data from global news feeds to build our knowledge base.

In [ ]:
from semantica.ingest import FeedIngestor

all_content = []
feeds = [
    "http://feeds.bbci.co.uk/news/world/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml"
]

feed_ingestor = FeedIngestor()
for f in feeds:
    try:
        data = feed_ingestor.ingest_feed(f)
        for item in data.items[:15]:
            text = item.content or item.description or item.title
            if text: all_content.append(text)
    except Exception: continue

clean_docs = [normalizer.normalize(text) for text in all_content if len(text) > 100]
chunks = []
for doc in clean_docs:
    chunks.extend(splitter.split(doc))

# Populate Stores
print(f"Processing {len(chunks)} chunks...")
embeddings = v_core.embedding_generator.generate_embeddings([str(c) for c in chunks])
vs.store_vectors(vectors=embeddings, metadata=[{"text": str(c)} for c in chunks])

# Build Knowledge Graph (Automatically persists to FalkorDB if connected)
kg = gb.build(sources=[{"text": str(c)} for c in chunks[:30]])

query = "Analyze the intersection of current regional conflicts and global energy infrastructure vulnerability."
print(f"\nSetup complete. Query: {query}")

## 3. Knowledge Refinement
Deduplicating entities and resolving conflicting information for higher accuracy.

In [ ]:
from semantica.deduplication import DuplicateDetector, EntityMerger
from semantica.conflicts import ConflictDetector, ConflictResolver

# 1. Entity Resolution
detector = DuplicateDetector(threshold=0.85)
merger = EntityMerger()
duplicates = detector.detect_duplicates(kg['entities'])
kg_refined = merger.merge_entities(kg, duplicates)

# 2. Conflict Resolution
conflict_detector = ConflictDetector()
conflicts = conflict_detector.detect_conflicts(kg_refined['entities'])
if conflicts:
    resolver = ConflictResolver()
    for conflict in conflicts:
        kg_refined = resolver.resolve_conflict(conflict, strategy="credibility_weighted")

print("Knowledge Graph Refined.")

## 4. Standard Vector RAG Answer
Standard RAG retrieves isolated chunks based on semantic similarity.

In [ ]:
q_vec = v_core.embedding_generator.generate_embeddings(query)
v_res = vs.search_vectors(q_vec, k=1)

print("--- VECTOR RAG FINAL ANSWER ---")
if v_res:
    print(v_res[0]['metadata']['text'])
else:
    print("No relevant context found.")

## 5. Semantica GraphRAG Answer
GraphRAG uses multi-hop traversal via **FalkorDB** to synthesize a connected answer.

In [ ]:
from semantica.context import ContextRetriever

# Use the persistent GraphStore if available, otherwise fallback to the in-memory dict
kg_backend = graph_store if use_persistent else kg_refined

retriever = ContextRetriever(
    vector_store=vs, 
    knowledge_graph=kg_backend, 
    use_graph_expansion=True,
    max_expansion_hops=2
)

g_res = retriever.retrieve(query, max_results=1)

print("--- GRAPHRAG FINAL ANSWER ---")
if g_res:
    print(g_res[0].content)
    if g_res[0].related_entities:
        print("\n--- CONNECTED ENTITIES DISCOVERED ---")
        for ent in g_res[0].related_entities[:5]:
            print(f"- {ent['content']} ({ent['type']}) via {ent['relationship']}")
else:
    print("No relevant multi-hop context found.")